In [15]:
# !! FINAL EMOTICON SENTIMENT CODE !!

# Google authentication key export
!export GOOGLE_APPLICATION_CREDENTIALS="/srv/Twitter2030/Data/acts-cloud-5-tljh-8e85a08eab1d-sakey.json"

from tweet_parser.tweet import Tweet
from tweet_parser.tweet_parser_errors import NotATweetError
import fileinput
import json
import string
import re
import time

# Imports the Google Cloud client library
from google.cloud import storage

# Create bucket client and authenticate via service account access key
storage_client = storage.Client.from_service_account_json('/srv/Twitter2030/Data/acts-cloud-5-tljh-8e85a08eab1d-sakey.json')

# Set bucket name, team folder name, and delimiter
BUCKET_NAME = "allteamsmlb" # Do not change
team_name = "NewTest" # Change as per team name directory, e.g., <##Name>

# Happy Emoticons
emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':c)', ':>', '=]', '=)', ':}', ':^)', ':-D',
    ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D', ':-))', ":'-)",
    ":')", ':^*', '>:P', ':-P', ':P', ':-p', ':p', '=p', ':-b', ':b', '>:)',
    '>;)', '>:-)', '<3', '☺️', '😚', '😙', '😋', '😛', '😜', '😝', '🤗', '😏',
    '😌', '🥳', '😎', '😺', '😸', '😹', '😻', '😼', '😽', '👍', '👏', '🙌',
    '😀', '😃', '😄', '😁', '😆', '😅', '🤣', '😂', '🙂', '😉', '😊', '😇',
    '🥰', '😍', '🤩', '😘', '🎉', '🎊', '💘', '💝', '💖', '💗', '💓', '💞',
    '💕', '💟', '❣️', '❤️‍🔥', '❤️', '🧡', '💛', '💚', '💙', '💜', '🤎', '🖤',
    '🤍', '💯'
    ])

# Sad Emoticons
emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', '>:\\', ';(', '😒', '😔', '🤢', '🤮', '🤧', '🥴', '😕', '😟', '🙁',
    '☹️', '😦', '😧', '😨', '😰', '😥', '😢', '😭', '😱', '😖', '😣', '😞',
    '😓', '😩', '😫', '😤', '😡', '😠', '🤬', '👿', '😿', '😾', '🖕', '👎',
    '🙍', '🙍‍♂️', '🙍‍♀️', '🙎', '🙎‍♂️', '🙎‍♀️', '🤦', '🤦‍♂️', '🤦‍♀️', '💔', '💢'
    ])

# Dictionaries: Key - Date    Value - Number of tweets of that sentiment
happy_sentiment_dict = {}
sad_sentiment_dict = {}
general_sentiment_dict = {}

print('El Fin')

El Fin


In [16]:
def clean_tweets(tweet):
	# remove stock market tickers like $GE
	tweet = re.sub(r'\$\w*', '', tweet)

	# remove old style retweet text "RT"
	tweet = re.sub(r'^RT[\s]+', '', tweet)

	# remove hyperlinks
	tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
	
	# remove hashtags
	# only removing the hash # sign from the word
	tweet = re.sub(r'#', '', tweet)
 
	return tweet

t0 = time.time()

object_generator = storage_client.list_blobs(BUCKET_NAME, prefix=team_name, delimiter=None)

for status_log_bucket_name in object_generator:
    #status_log_gz = status_log_bucket_name.name.split('/')
    #print(status_log_bucket_name)
    #print(status_log_gz[1])

# https://googleapis.dev/python/storage/latest/blobs.html#google.cloud.storage.blob.Blob.download_as_text
# See section on open() function about 70% down the page
    client2 = storage.Client()
    BUCKET_NAME_TWO = client2.bucket(BUCKET_NAME)
    blob = BUCKET_NAME_TWO.blob(status_log_bucket_name.name)
    #blob = BUCKET_NAME.blob(status_log_gz)

#blobcontent = blob.download_as_string()

    with blob.open('rt') as f:
        for line in f:
            try:
                tweet_dict = json.loads(line)

                tweet = Tweet(tweet_dict)
                
                text = clean_tweets(tweet.all_text)
                                
                # Extracts date and verifies its a day the Sox play
                date = tweet.created_at_string[0:10]
                 
                # if date in sox_game_dates:
                for emoticon in emoticons_happy:
                    if text.lower().find(emoticon) != -1:
                      # There's a happy emoticon: Adds to the dict
                      if date in happy_sentiment_dict.keys():
                        happy_sentiment_dict[date] += 1
                      else:
                        happy_sentiment_dict[date] = 1
                      if date in general_sentiment_dict.keys():
                        general_sentiment_dict[date] += 1
                      else:
                        general_sentiment_dict[date] = 1
                      break;

                for emoticon in emoticons_sad:
                    if text.lower().find(emoticon) != -1:
                      # There's a sad emoticon: Adds to the dict
                      if date in sad_sentiment_dict.keys():
                        sad_sentiment_dict[date] += 1
                      else:
                        sad_sentiment_dict[date] = 1
                      if date in general_sentiment_dict.keys():
                        general_sentiment_dict[date] += 1
                      else:
                        general_sentiment_dict[date] = 1
                      break;

            except (json.JSONDecodeError, NotATweetError):
                pass
            
t1 = time.time() 
            
print('El Fin')
print('Runtime:', str(round(t1 - t0, 2)), 'seconds')

El Fin
Runtime: 1.36 seconds


In [14]:
with open("Output/Sentiment/TEST_happy.txt", 'w') as new_file: 
    for key, value in happy_sentiment_dict.items(): 
        new_file.write('%s : %d\n' % (key, value))

with open("Output/Sentiment/TEST_sad.txt", 'w') as new_file: 
    for key, value in sad_sentiment_dict.items(): 
        new_file.write('%s : %d\n' % (key, value))

with open("Output/Sentiment/TEST_general.txt", 'w') as new_file: 
    for key, value in general_sentiment_dict.items(): 
        new_file.write('%s : %d\n' % (key, value))
        
print('El Fin')

El Fin
